## Value-based methods
### 1. DQN

In [25]:
# discretizing actions
a_high, a_low = env.action_space.high, env.action_space.low
n_actions = np.array([a_low, np.array([0, 0.5, 0.5]), a_high])

In [26]:
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()
        
    def forward(self, x):
        return x.view(x.size(0), -1)

In [27]:
class network(nn.Module):
    def __init__(self):
        super(network, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(3, 3), stride=2)
        self.conv2 = nn.Conv2d(16, 32, 3, 2)
        self.conv3 = nn.Conv2d(32, 64, 3, 2)
        self.conv4 = nn.Conv2d(64, 128, 3, 2) # (128, 5, 5)
        self.flatten = Flatten()
        self.dense1 = nn.Linear(128*5*5, 800)
        self.dense2 = nn.Linear(800, n_actions.shape[0])
        
    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.ReLU()(self.conv2(x))
        x = nn.ReLU()(self.conv3(x))
        x = nn.ReLU()(self.conv4(x))
        x = nn.Tanh(self.dense1(x))
        x = self.dense2(x)
        
        return x

In [28]:
agent = network()
target_network = network()

In [42]:
actions = torch.tensor([[-1.,  0.,  0.], [1., 1., 1.], [0. , 0.5, 0.5], [-1.,  0.,  0.]])
actions.shape

torch.Size([4, 3])

In [66]:
def to_onehot(vectors):
    vectors = vectors.data.numpy()
    unique_vectors = np.unique(vectors, axis=0)
    print(unique_vectors)
    dict_unique_vectors = {i: vector for i, vector in enumerate(unique_vectors)}
    N = unique_vectors.shape[0] # amount of vectors
    print(N, vectors.shape[0])
    onehots = np.zeros((vectors.shape[0], N))
    for i, vector in enumerate(vectors):
        index = [i for i, vector_ in dict_unique_vectors.items() if (vector_==vector).all() ]
        print(index)
        onehots[i][index] = 1
        
    return torch.tensor(onehots, dtype=torch.float32)

In [ ]:
def compute_loss(states, actions, rewards, dones, next_states, eps=0.5):
    states = torch.tensor(states, dtype=torch.float32)
    actions = torch.tensor(actions, dtype=torch.float32)
    next_states = torch.tensor(next_states, dtype=torch.float32)
    rewards = torch.tensor(rewards, dtype=torch.float32)
    dones = torch.tensor(dones)
    
    pred_qvalues = agent(states)
    q_values_for_actions = torch.sum(pred_qvalues*to_onehot(actions), dim=0)
    
    next_pred_qvalues = target_network(next_states)